# Assignment 3 - Data Science in Finance and Insurance
Submission by: Dennis Goldenberg

In [63]:
#import statements
import numpy as np;
import pandas as pd;
import plotnine as pn;

#set constants
A,B,c = 2.2e-4,2.7*np.exp(1) - 6,1.124

(A,B,c)

(0.00022, 1.3393609368394221, 1.124)

## Part [a] - Repeated Simpson's Rule for integration in Numpy

### Density function for future life Expectancy
As stated in the problem, I assume that mortality follows the Makeham law, or:
$$f_{T_x}(t) = (A + Bc^{x + t})e^{-At -\frac{B}{\log(c)}c^x(c^t - 1)}$$
Here, $x$ is the current age of the person, $t$ is the number of years they have left to live, and $A,B,c$ are constants, as defined above. I code this function below.

In [64]:
#Equation for probability density of living t more years given current age x
Makeham = lambda t,x,A = A,B = B,c = c : \
    (A + B*(c**(x + t)))*np.exp(-1*A*t - (B/np.log(c))*(c**x)*((c**t)- 1))

#Create version multiplied by t for purposes of expectation
multMakeham = lambda t,x,A = A,B = B,c = c : \
      t * (A + B*(c**(x + t)))*np.exp(-1*A*t - (B/np.log(c))*(c**x)*((c**t)- 1))


In [66]:
#Function for simpson's Method on Makeham, given the age
def Simpsons(func, currAge, a = 0, n = 50):
    #set upper bound of integration
    b = 130 - currAge;
    h = (b - a)/(2*n)
    
    #create arrays for function input for I
    input_1 = np.array([a + (2*j - 1)*h for j in range(1, n + 1)])
    input_2 = np.array([a + 2*j*h for j in range(1, n)])

    #Simpson's Rule approximation
    I = (h/3)*(func(a, currAge) + 4 * np.sum([func(t, currAge) for t in input_1]) +\
                2 * np.sum([func(t, currAge) for t in input_2]) + func(b, currAge))
    return I;

Simpsons(multMakeham,0)

0.6255507337986156